        El IPC o Indice de precios al consumidor es un indice economico en el que se valoran los precios de un determinado conjunto de bienes y servicios determinado sobre la base de la encuesta continua de presupuestos familiares. 
        Si el porcentaje es positivo indica una subida de los precios, mientras que si es negativo una bajada

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.express as px
from pyspark.sql import functions as f
import plotly.graph_objects as go

In [ ]:
from pyspark.sql import SparkSession

my_spark = SparkSession \
    .builder \
    .appName("PySparkShell") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/castillaYLeon.IPC") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/castillaYLeon.IPC") \
    .getOrCreate()


In [ ]:
df = spark.read.format("mongo").load()
df = df.drop("_id")
df = df.withColumn("MES",f.to_date(df.MES,'MM-yyyy').alias('dt'))
df = df.withColumn("Año",f.year(df.MES))
df = df.orderBy("MES")

In [ ]:
dfIPC = df.toPandas()

In [ ]:
figIPCAnualLine = go.Figure(data=[
    go.Scatter(name="IPC-VARIACIÓN ANUAL(CyL)", x=dfIPC.MES , y=dfIPC["PRECIOS - INDICE PRECIOS CONSUMO (IPC) - BASE 2016 - VARIACIÓN ANUAL(CyL)"], mode='lines+markers'),
    go.Scatter(name="IPC-VARIACIÓN ANUAL(España)", x=dfIPC.MES , y=dfIPC["PRECIOS - INDICE PRECIOS CONSUMO (IPC) - BASE 2016 - VARIACIÓN ANUAL(España)"],mode='markers')
]                            )
#el slider de la grafica pequeño junto al rango que permite seleccionar mas el boton de step por año
# esto puede ser utilizado para una grafica de express(px) debido a que es un update_layout
figIPCAnualLine.update_layout(
                            xaxis=dict(
                                rangeselector=dict(
                                    buttons=list([
                                        dict(count=1,
                                             step="year",
                                             stepmode="todate"),
                                        
                                    ])
                                ),
                                rangeslider=dict(
                                    visible=True
                                ),
    ),
    #title = "Grafica IPC Anual en linea", #Este no seria necesario pues lo rehago mas abajo pero asi muestro como hacerlo simple aunque sin style
    xaxis_title = "Periodo",
    yaxis_title = "Indice Precios",
    legend_title="Precios",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
    title={
        'text': "Grafica IPC Anual en linea",
        'y': 0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

figIPCAnualLine.show()

In [ ]:
figIPCAnualBarras = go.Figure()

# Add traces, one for each slider step
for years in dfIPC["Año"].unique(): #recorro un bucle con cada año
    aux = dfIPC[dfIPC["Año"]==years] #Creo un aux que sera el df con el año que necesito
     
    figIPCAnualBarras.add_bar( #añado las barras de cyl correspondientes a este año
        visible=False, #las ocultamos
        name="IPC-VARIACIÓN ANUAL(CyL)",
        x=aux.MES,
        y=aux["PRECIOS - INDICE PRECIOS CONSUMO (IPC) - BASE 2016 - VARIACIÓN ANUAL(CyL)"]
    )
    figIPCAnualBarras.add_bar( #añado las barras de españa correspondientes a este año 
        visible=False, # las ocultamos
        name="IPC-VARIACIÓN ANUAL(España)",
        x=aux.MES,
        y=aux["PRECIOS - INDICE PRECIOS CONSUMO (IPC) - BASE 2016 - VARIACIÓN ANUAL(España)"]
    )
        

# Hacemos que la 1º y la 2º barra de la figura sean visibles, de esta forma nada mas cargue el grafico nos los mostrara por pantalla.
figIPCAnualBarras.data[0].visible = True
figIPCAnualBarras.data[1].visible = True
# Create and add slider
steps = []
#steps, son los pasos , los cambios que hay entre cada año
# es en los steps donde hacemos visibles las barras y actualizamos la grafica
for i in range(0,len(figIPCAnualBarras.data),2): # recorro un bucle con todos los datos de la grafica, hago una iteracion de 2 en 2 debido a que me interesa sacar dos datos cada vez, dos barras
    step = dict(
        method="update", 
        args=[{"visible": [False] * len(figIPCAnualBarras.data)},  
              {"title": "Grafica IPC año: " + str(dfIPC.Año.unique().tolist()[int(i/2)])}] # layout attribute y nombre de la grafica
    )
    
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible", hacemos visible esta barra
    step["args"][0]["visible"][i+1] = True #y esta tambien pues necesitamos hacer visibles a las dos barras
    
    steps.append(step)
#sliders
sliders = [dict(
    active=dfIPC["Año"].nunique(),
    currentvalue={"prefix":"Año: "}, #el prefijo que aparece entre la grafica y justo encima del slider
    pad={"t": dfIPC["Año"].nunique()}, #el numero de steps que tendra el slider
    steps=steps
)]

##
#añado los sliders y estilizo un poco el grafico 
figIPCAnualBarras.update_layout(
    sliders=sliders,
    xaxis_title = "Periodo",
    yaxis_title = "Indice Precios",
    legend_title="Precios",
    font=dict(
        color="brown"
    ),
    title={
        'text': "Grafica IPC Anual en Barra",
        'y': 0.9,
        'x':0.4,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

##
# Edit slider labels
figIPCAnualBarras['layout']['sliders'][0]['currentvalue']['prefix']='Año: '
for i, date in enumerate(dfIPC["Año"].unique().tolist(), start = 0):
#for i,date in enumerate(lista, start = 0):
    figIPCAnualBarras['layout']['sliders'][0]['steps'][i]['label']= date
    

figIPCAnualBarras.show()

In [ ]:
figIPCAnualLine.write_html("C:/Users/david/OneDrive/Imágenes/GraficosPlotly/Economia/IPC/FiguraIPCDiferenciaEspCyLLinea.html")
figIPCAnualBarras.write_html("C:/Users/david/OneDrive/Imágenes/GraficosPlotly/Economia/IPC/FiguraIPCDiferenciaEspCyLBarra.html")